In [28]:
#region Chargement des packages
import sys
import os
if os.path.basename(os.getcwd())=='Prospective_conso':
    sys.path.append('../../')
from functions.f_notebook import hide_toggle
import pandas as pd
from functions.f_tools import *
from functions.f_graphicalTools import *
from Models.Prospective_conso.f_evolution_tools import *
import qgrid # great package https://github.com/quantopian/qgrid
from mycolorpy import colorlist as mcp
import numpy as np
import time
from functools import partial
dpe_colors = ['#009900', '#33cc33', '#B3FF00', '#e6e600', '#FFB300', '#FF4D00', '#FF0000',"#000000"]
Graphic_folder = "Graphics/"
Data_folder = "data/"
#endregion

# Modélisation de l'évolution de la consommation et des émissions dans le secteur du transport en France

## Introduction <a class="anchor" id="intro"></a>

### Table of Contents

* [1. Introduction](#intro)
* [2. Hypothèses de modélisation du transport de voyageurs](#initialstate)
* [3. Hypothèses de modélisation du fret](#decarbonization)
* [4. Simulation](#simulation)
    * [4.1 Etat actuel du parc](#actuel)
    * [4.2 Résultats de la simulation prospective](#prospective)
* [5. Biblioraphie](#biblio)

## 2. Hypothèses de modélisation du transport de voyageurs <a class="anchor" id="initialstate"></a>

In [29]:
#region chargement des données voyageurs
#start = time.process_time()
dim_names=["Categorie","year","Vecteur"];
Index_names = ["Categorie"];Energy_system_name="Categorie"
data_set_from_excel =  pd.read_excel(Data_folder+"Hypotheses_Transport_1D.xlsx", None);
sim_param = extract_sim_param(data_set_from_excel,Index_names = Index_names,dim_names=dim_names,Energy_system_name=Energy_system_name)
sim_param["init_sim_stock"]=create_initial_parc(sim_param).sort_index()
sim_param["volume_variable_name"] = "Mds_voy_km"
sim_param["retrofit_improvement"]=pd.DataFrame([sim_param["retrofit_improvement"]]*len(sim_param['base_index_year']),index =sim_param['base_index_year'])[0]
Para_2_fill = {param : sim_param["base_index_year"] for param in ["retrofit_change_Mds_voy_km","retrofit_Transition"]}
sim_param = interpolate_sim_param(sim_param)
sim_param["retrofit_change_Mds_voy_km"]=sim_param["retrofit_change_total_proportion_Mds_voy_km"].diff().fillna(0)
sim_param   =   complete_parameters(sim_param,Para_2_fill=Para_2_fill)
sim_param["retrofit_change_Mds_voy_km"]=sim_param["retrofit_change_Mds_voy_km"]*sim_param["init_sim_stock"]["Mds_voy_km"]
sim_param["init_sim_stock"]["energy_need_per_Mds_voy_km"] = sim_param["init_sim_stock"].apply(
                            lambda x: 1 /(100*x["remplissage"]), axis=1).fillna(0)
sim_param=set_model_functions(sim_param)
sim_param["retrofit_Transition"]=sim_param["retrofit_Transition"].fillna(0)
sim_param_voyageurs = sim_param
#end = time.process_time()
#print("Chargement des données, des modèles et interpolation terminés en : "+str(end-start)+" secondes")
#endregion

## 3. Hypothèses de modélisation du fret <a class="anchor" id="decarbonization"></a>

In [30]:
dim_names=["Categorie","year","Vecteur"];
Index_names = ["Categorie"];Energy_system_name="Categorie"
data_set_from_excel =  pd.read_excel(Data_folder+"Hypotheses_Transport_Fret_1D.xlsx", None);
sim_param = extract_sim_param(data_set_from_excel,Index_names = Index_names,dim_names=dim_names,Energy_system_name=Energy_system_name)

In [31]:
## for those able to make qgrid work, you should be able to modify the table sim_param['retrofit_Transition'] within the interface below
#https://github.com/quantopian/qgrid
col_def = { col :  { 'editable': True } for col in sim_param['retrofit_Transition'].columns}
qgrid_widget = qgrid.show_grid(sim_param['retrofit_Transition'], show_toolbar=True, column_definitions=col_def)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
sim_param['retrofit_Transition']=qgrid_widget.get_changed_df()

In [33]:
#region chargement des données fret
#start = time.process_time()

sim_param["init_sim_stock"]=create_initial_parc(sim_param).sort_index()
sim_param["volume_variable_name"] = "Mds_t_km"
sim_param["retrofit_improvement"]=pd.DataFrame([sim_param["retrofit_improvement"]]*len(sim_param['base_index_year']),index =sim_param['base_index_year'])[0]
Para_2_fill = {param : sim_param["base_index_year"] for param in ["retrofit_change_Mds_t_km","retrofit_Transition"]}
sim_param = interpolate_sim_param(sim_param)
sim_param["retrofit_change_Mds_t_km"]=sim_param["retrofit_change_total_proportion_Mds_t_km"].diff().fillna(0)
sim_param   =   complete_parameters(sim_param,Para_2_fill=Para_2_fill)
sim_param["retrofit_change_Mds_t_km"]=sim_param["retrofit_change_Mds_t_km"]*sim_param["init_sim_stock"]["Mds_t_km"]
sim_param["init_sim_stock"]["energy_need_per_Mds_t_km"] = sim_param["init_sim_stock"].apply(lambda x: 1 /(100), axis=1).fillna(0)
sim_param=set_model_functions(sim_param)
sim_param["retrofit_Transition"]=sim_param["retrofit_Transition"].fillna(0)
sim_param_fret = sim_param
#end = time.process_time()
#print("Chargement des données, des modèles et interpolation terminés en : "+str(end-start)+" secondes")
#endregion

## 4.Simulation <a class="anchor" id="simulation"></a>

In [34]:
#region simulation
sim_stock_voyageurs = launch_simulation(sim_param_voyageurs)
sim_stock_df_voyageurs = pd.concat(sim_stock_voyageurs, axis=0).reset_index().\
    rename(columns={"level_0":"year"}).set_index([ "year"  ,  Energy_system_name  , "old_new"])
sim_stock_fret = launch_simulation(sim_param_fret)
sim_stock_df_fret = pd.concat(sim_stock_fret, axis=0).reset_index().\
    rename(columns={"level_0":"year"}).set_index([ "year"  ,  Energy_system_name  , "old_new"])
sim_stock_df = pd.concat([sim_stock_df_voyageurs,sim_stock_df_fret])
#endregion

Computing: [########################################] 29/29

Computing: [########################################] 29/29



### 4.1 Situation initiale <a class="anchor" id="actuel"></a>

In [35]:
### categories pour avoir des groupes de couleurs dans les graphiques
col_class_dict={'Bus et cars GNV' : 1, 'Bus et cars H2':1, 'Bus et cars diesel':1, 'Bus et cars électrique':1,
 'Rail court':2, 'Rail long':2, 'Camion fret' : 2,'camion fret H2' : 2,
                'VP GNV':3, 'VP fuel':3, 'VP électrique':3,
 'VUL GNV':4, 'VUL fuel':4, 'VUL électrique':4,
 'aerien_international':5, 'aerien_interne':5, 'aerien_outre_mer':5,
 'deux roues diesel':6, 'deux roues électrique':6,
 'Avion fret  international': 7, 'Bateau fret  international': 7,'Train fret':7}

import plotly.express as px
Var = "Conso"
y_df = sim_stock_df.loc[(2021,slice(None),slice(None))].groupby([Energy_system_name])[Var].sum().to_frame().reset_index()
#y_df.loc[:,"Categorie"]=pd.MultiIndex.from_tuples([(str(col_class_dict[key]),key) for key in y_df.Categorie])
color_dict = gen_grouped_color_map(col_class_dict)
y_df["class"]=[col_class_dict[cat] for cat in y_df["Categorie"]]
y_df=y_df.sort_values(by=['class'])
fig = px.bar(y_df,x="class", y=Var, color="Categorie", title="Wide-Form Input",color_discrete_map=color_dict)
fig=fig.update_layout(title_text="Conso énergie finale par mode de transport (en TWh)", xaxis_title="Categorie",yaxis_title="Conso [TWh]")
fig.show()

In [36]:
Var = "emissions"
y_df = sim_stock_df.loc[(2021,slice(None),slice(None))].groupby([Energy_system_name])[Var].sum().to_frame().reset_index()
#y_df.loc[:,"Categorie"]=pd.MultiIndex.from_tuples([(str(col_class_dict[key]),key) for key in y_df.Categorie])
color_dict = gen_grouped_color_map(col_class_dict)
y_df["class"]=[col_class_dict[cat] for cat in y_df["Categorie"]]
y_df=y_df.sort_values(by=['class'])
fig = px.bar(y_df,x="class", y=Var, color="Categorie", title="Wide-Form Input",color_discrete_map=color_dict)
fig=fig.update_layout(title_text="Emissions par mode de transport (en MTCO2)", xaxis_title="Categorie",yaxis_title="Emissions [MTCO2]")
fig.show()

### 4.2 Résultats de la simulation prospective <a class="anchor" id="prospective"></a>

In [37]:
#region représentations graphiques

Var = "Conso"
y_df = sim_stock_df.groupby(["year",Energy_system_name])[Var].sum().to_frame().reset_index().\
    pivot(index=['year'], columns=Energy_system_name).loc[[year for year in sim_param["years"][1:]],Var]
y_df.columns=pd.MultiIndex.from_tuples([(str(col_class_dict[key]),key) for key in y_df.columns])

fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Conso énergie finale par mode de transport (en TWh)", xaxis_title="Année",yaxis_title="Conso [TWh]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

#endregion

In [38]:
Var = "emissions"
y_df = sim_stock_df.groupby(["year",Energy_system_name])[Var].sum().to_frame().reset_index().\
    pivot(index=['year'], columns=Energy_system_name).loc[[year for year in sim_param["years"][1:]],Var]/10**3
y_df.columns=pd.MultiIndex.from_tuples([(str(col_class_dict[key]),key) for key in y_df.columns])

fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Emissions de GES par mode de transport (en MtCO2e)", xaxis_title="Année",yaxis_title="Conso [MtCO2e]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()


In [39]:
y_df = sim_stock_df.groupby(["year"])[[ 'conso_'+Vecteur for Vecteur in sim_param["Vecteurs"]]].sum().loc[[year for year in sim_param["years"][1:]],:]
fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Conso énergie finale par vecteur (en TWh)", xaxis_title="Année",yaxis_title="Conso [TWh]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

In [40]:
y_df = sim_stock_df.groupby(["year"])[[ 'emissions_'+Vecteur for Vecteur in sim_param["Vecteurs"]]].sum().loc[[year for year in sim_param["years"][1:]],:]
fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Emissions par vecteur [MT CO2]", xaxis_title="Année",yaxis_title="CO2 [MT]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

## 5. Bibliographie <a class="biblio" id="initialstate"></a>

- [Guide très complet du gouvernement](https://www.ecologie.gouv.fr/sites/default/files/Info%20GES_Guide%20m%C3%A9thodo.pdf) donne les consommations et les émissions pour toutes les sortes de transports et les analyses qui permettent d'obtenir ces valeurs.

- [Les chiffres clés du transport](https://www.statistiques.developpement-durable.gouv.fr/edition-numerique/chiffres-cles-transports-2022/) du SDES

- [Données et études statistiques du SDES sur le transport](https://www.statistiques.developpement-durable.gouv.fr/transports)

- [Etude Prospective de France stratégie](https://www.strategie.gouv.fr/sites/strategie.gouv.fr/files/atoms/files/prospective_2040-2060_des_transports_et_des_mobilites_-_rapport_de_synthese_-_fevrier_2022.pdf) février 2022

- [Etude prospective de l'ADEME](https://librairie.ademe.fr/cadic/6531/transitions2050-rapport-compresse.pdf?modal=false) "transition 2050" le transport est p172

- [Article sur le transport maritime mondial](https://www.polytechnique-insights.com/dossiers/energie/les-innovations-bas-carbone-du-fret-maritime/comment-reduire-lempreinte-carbone-du-fret-maritime/#note-4)

- [Rapport parlementaire de 2021](http://www.senat.fr/rap/r20-604/r20-604_mono.html) Transport de marchandises face aux impératifs environnementaux

- [Un document du lobby de l'avition](https://protectiondelaerien.fr/info-ou-infox-lavion-est-il-un-moyen-de-transport-efficace/) pas inintéressant. Le problème n'est pas l'inefficacité de l'avion mais la démesure du besoin de distance/vitesse.
